In [37]:
from dataclasses import dataclass, field
from typing import Optional, Union, get_args, Dict

@dataclass
class TradeOrder:
    symbol: str
    tradeTimestamp: int
    entry_price: Union[float, int]
    position: int
    quantity: float
    leverage: int
    fee_rate: float = 0.05
    current_price: Optional[Union[float, int]] = None
    break_event_price: Optional[float] = None
    fee_open: Optional[Union[float, int]] = None
    fee_close: Optional[Union[float, int]] = None
    memo: Optional[str] = None

    def __post_init__(self):
        # 자동화된 타입 검증
        for field in fields(self):
            value = getattr(self, field.name)
            if value is not None:
                if hasattr(field.type, '__origin__'):  # 복합 타입 (e.g., Union)
                    expected_types = get_args(field.type)
                else:  # 기본 타입 (e.g., str, int, float)
                    expected_types = (field.type,)
                
                if not isinstance(value, expected_types):
                    raise ValueError(f"{field.name} 값 입력 오류: {value} / 기대 타입 {expected_types}")
        
        # current_price 초기값 설정
        if self.current_price is None:
            self.current_price = self.entry_price
        
        # 레버리지 값 검증
        if self.leverage <= 0:
            raise ValueError(f"레버리지는 최소 1 이상이어야 합니다. 현재 값: {self.leverage}")
        
        # 초기 수수료 계산
        self.__update_fee()
    
    def __update_fee(self):
        """수수료를 계산하여 업데이트"""
        adjusted_fee_rate = self.fee_rate / 1_000  # self.fee_rate 사용
        self.fee_open = self.entry_price * adjusted_fee_rate * self.quantity
        self.fee_close = self.current_price * adjusted_fee_rate * self.quantity
        self.break_event_price = self.entry_price + self.fee_open

    def update_current_price(self, current_price: float):
        """현재 가격 업데이트 및 수수료 재계산"""
        self.current_price = current_price
        self.__update_fee()
    
    def __repr__(self):
        """사용자 정의 출력 형식"""
        return (
            f"TradeOrder(symbol='{self.symbol}', tradeTimestamp={self.tradeTimestamp}, "
            f"entry_price={self.entry_price:.6f}, position={self.position}, quantity={self.quantity}, "
            f"leverage={self.leverage}, fee_rate={self.fee_rate}, current_price={self.current_price:.6f}, "
            f"break_event_price={self.break_event_price}, fee_open={self.fee_open:.6f}, "
            f"fee_close={self.fee_close:.6f}, memo={self.memo})"
        )

In [38]:
@dataclass(slots=True)
class DynamicDataStore:
    intervals: dict = field(default_factory=dict)  # 동적 간격 관리

# 사용 예시
data_store = DynamicDataStore()
data_store.intervals['1m'] = {"timestamp": 1630000000, "value": "1m_data"}
print(data_store.intervals['1m'])  # {'timestamp': 1630000000, 'value': '1m_data'}

{'timestamp': 1630000000, 'value': '1m_data'}


In [29]:
xrpusdt_.update_current_price(current_price=1.15)

In [31]:
xrpusdt_

TradeOrder(symbol='XRPUSDT', tradeTimestamp=12, entry_price=1.256300, position=5, quantity=9.6, leverage=5, fee_rate=0.05, current_price=1.150000, break_event_price=1.2569030239999999, fee_open=0.000603, fee_close=0.000552, memo=None)

In [ ]:
>>> asyncio.run(obj.submit_order(symbol='ADAUSDT', side='SELL', price=1.015, quantity=5, order_type='STOP_MARKET'))
{'orderId': 46915674924, 'symbol': 'ADAUSDT', 'status': 'NEW', 'clientOrderId': '8GSV2rVnGLMBnO4HDwSb1Y', 'price': '0.00000', 'avgPrice': '0.00', 'origQty': '5', 'executedQty': '0', 'cumQty': '0', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'STOP_MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '1.01500', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'STOP_MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1733915429075}
>>> asyncio.run(obj.submit_order(symbol='ADAUSDT', side='SELL', quantity=5, order_type='MARKET'))
{'orderId': 46916207450, 'symbol': 'ADAUSDT', 'status': 'NEW', 'clientOrderId': '476bSE3TUnGfOp6AsVCq12', 'price': '0.00000', 'avgPrice': '0.00', 'origQty': '5', 'executedQty': '0', 'cumQty': '0', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '0.00000', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1733916764850}
>>>